This file is written to compare the epd fasta file that contain sequences of promoters, filtered based on capable to find corresponding Ensembl ID. Then this is compared with the RNA-seq file that contain RNA-seq data for a single sample, where the RNA-seq matrix is filtered based on corresponding ENsembl ID from epd fasta file.

In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/master_thesis/inputs

/content/drive/MyDrive/master_thesis/inputs


In [3]:
try:
    import google.colab
    # Running on Google Colab, so install Biopython first
    !pip install biopython
except ImportError:
    pass

     |████████████████████████████████| 2.3 MB 5.1 MB/s 


In [4]:
try:
    import google.colab
    # Running on Google Colab, so install Biopython first
    !pip install mygene
except ImportError:
    pass

In [5]:
from Bio import SeqIO
import csv
import mygene
mg = mygene.MyGeneInfo()
import pandas as pd
from google.colab import data_table
import numpy as np
import os

In [6]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

pd.set_option('display.max_columns', None)

In [7]:
def epdparser(fasta_file):
  """convert gene symbol/ID to Ensembl gene ID

  Args:
  fasta_file = txt file containing fasta records from epd promoter database
  output:
  epd_pd = dataframe of 2 columns Promoter_ID, Sequence
  """
  Promoter_seq_pairs = []
  with open(fasta_file) as handle:
    for value in SeqIO.FastaIO.SimpleFastaParser(handle):
      Promoter_ID = value[0].split()[1][:-2]
      seq = value[1]
      Promoter_seq_pairs.append([Promoter_ID, seq])
  epd_df = pd.DataFrame(Promoter_seq_pairs, columns = ("Promoter_ID",\
                                                   "Sequence"))
  return epd_df

In [8]:
def genesymboltoensemblid(epd_df):
  """convert gene symbol/ID to Ensembl gene ID

  input:
  epd_df = dataframe of two columns, Promoter_ID and Sequence
  output:
  matching_df = dataframe of three columns gene_id, Ensembl_ID, 
               and Sequence. Contain respective gene name, Ensembl_ID, 
              sequence of a promoter.
  """
  match_list = []
  genesymbol_list = epd_df["Promoter_ID"].tolist()
  seq_list = epd_df["Sequence"].tolist()
  output = mg.querymany(genesymbol_list, scopes='symbol'\
                        , fields='ensembl.gene', species='human', returnall=False)
  for record in output:
    gene_sym, ensembl_ids= None, None
    #when query does not have corresponding ensembl id
    if 'notfound' in record.keys():
      if record["notfound"]:
        pass
    else:
      gene_sym = record["query"]
      seq = seq_list[genesymbol_list.index(gene_sym)]
      #ensembl id returns sometimes as a list of dicts when multiple ensembl
      #ids were matched to
      if "ensembl" in record.keys():
        ensembl_ids = record["ensembl"]
        #select first ensembl id from list of dicts
        if type(ensembl_ids) is list:
          ensembl_id = ensembl_ids[0]["gene"]
        else:
          ensembl_id = ensembl_ids["gene"]
      match_list.append([gene_sym, ensembl_id, seq])
    match_df = pd.DataFrame(match_list, columns = ("gene_id",\
                                                   "Ensembl_ID", "Sequence"))
  return match_df

In [9]:
def remove_version(Ensembl_ID):
  """remove version number from gene_id

  Args:
  gene_id = string, contain gene_id and version after .
  output:
  epd_pd = string, contain gene_id
  """
  if "." in Ensembl_ID:
    Ensembl_ID = Ensembl_ID.split(".", 1)[0]
  return Ensembl_ID

def RNAseqparser(RNAseqfile):
  """Parse RNAseq matrix to extract relevant columns

  Args:
  RNAseqfile = csv file containing RNA-seq data
  output:
  rd = dataframe of three columns, gene_id, TPM, FPKM
  """
  rd = pd.read_csv(open(RNAseqfile), delimiter="\t", quotechar='"', \
                    usecols = ["gene_id", "TPM", "FPKM"])
  #remove different versions from gene_id column
  rd["gene_id"] = rd["gene_id"].apply(remove_version)
  return rd

In [10]:
def RNAseq_matches(RNA_seq_matrix, matching_df):
  """compare if rows in RNA_seq_matrix has gene_id that is corrispond to 
      matching_df's Ensembl_ID, these rows are sublisted and
      duplicates are removed

  Args:
  RNA_seq_matrix = dataframe of three columns, gene_id, TPM, FPKM
  matching_df = dataframe of three columns gene_id, Ensembl_ID, 
               and Sequence. Contain respective gene name, Ensembl_ID, 
              , sequence of a promoter.
  output:
  rd = dataframe of three col, Ensembl_ID, TPM, FPKM
  """
  matching_list = matching_df["Ensembl_ID"].tolist()
  sub_matrix = RNA_seq_matrix[RNA_seq_matrix["gene_id"].isin(matching_list)]
  return sub_matrix

In [11]:
def joinsequenceapply(match_df, parsed_matrix):
  """join the match_df and parsed_matrix based on the ensembl ID of each
      record, to link the RNA-Seq data to the Sequence

  Args:
  parsed_matrix = dataframe of three columns, gene_id, TPM, FPKM.
  matching_df = dataframe of three columns gene_id, Ensembl_ID, 
               and Sequence. Contain respective gene name, Ensembl_ID, 
              , sequence of a promoter.
  output:
  joined_matrix = dataframe of 5 col, Ensembl_ID, TPM, FPKM, gene_id and Sequence, each row
        contain information of the 5 categories of a single promoter and its
        respective gene
  """
  gene_ids = parsed_matrix["gene_id"].tolist()
  #filter match_df with those only contain ensembl_ID in parsed_matrix
  match_df_Ensembl_ID = match_df[match_df["Ensembl_ID"].isin(gene_ids)]
  renamed_parsed_matrix = parsed_matrix.rename({"gene_id": "Ensembl_ID"}, axis=1)
  joined_matrix = pd.merge(renamed_parsed_matrix, match_df_Ensembl_ID, on=["Ensembl_ID"])
  return joined_matrix

In [12]:
def wrapperparser(epd_fasta_file, RNA_seq_file):
    """
    Wrapper that combine RNA-seq file and epd promoter files

    Arg:
    epd_fasta_file = fasta file generated epd promoter database
    RNA_seq_file = tsv file that contain the RNA_seq data of a sample

    Output:
    joined_matrix = dataframe of 5 col, Ensembl_ID, TPM, FPKM, gene_id and Sequence, each row
          contain information of the 5 categories of a single promoter and its
          respective gene. 
    """
    name = epd_fasta_file[:-4] + "_" + RNA_seq_file[:-4]
    if os.path.isdir(name):
        pass
    else:
        os.mkdir(name)
    tsv_name = os.path.join(name, name + ".csv")
    Promoter_seq_pd = epdparser(epd_fasta_file)
    match_df = genesymboltoensemblid(Promoter_seq_pd)
    RNA_seq_matrix = RNAseqparser(RNA_seq_file)
    parsed_matrix = RNAseq_matches(RNA_seq_matrix, match_df)
    joined_matrix = joinsequenceapply(match_df, parsed_matrix)
    joined_matrix['TPM'] = joined_matrix['TPM'] + 1
    joined_matrix['logTPM'] = np.log2(joined_matrix['TPM'])
    joined_matrix['FPKM'] = joined_matrix['FPKM'] + 1
    joined_matrix['logFPKM'] = np.log2(joined_matrix['FPKM'])
    if os.path.isfile(tsv_name) == False:
        joined_matrix.to_csv(tsv_name)

In [ ]:
#wrapperparser("hg38_msxTm.txt", "ENCFF910TAZ.tsv")
#wrapperparser("hg38_msxTm.txt", "ENCFF292FVY.tsv")
#wrapperparser("hg38_msxTm.txt", "ENCFF279XTY.tsv")
#wrapperparser("hg38_msxTm.txt", "ENCFF597FGD.tsv")
wrapperparser("hg38_classification_1.txt", "ENCFF910TAZ.tsv")
wrapperparser("hg38_classification_0.txt", "ENCFF910TAZ.tsv")

In [48]:
def wrapperparser_plant(epd_fasta_file, GSE_RNA_seq_file):
    """
    Wrapper that combine RNA-seq file and epd promoter files

    Arg:
    epd_fasta_file = fasta file generated epd promoter database
    RNA_seq_file = tsv file that contain the RNA_seq data of a sample

    Output:
    joined_matrix = dataframe of 5 col, Ensembl_ID, TPM, FPKM, gene_id and Sequence, each row
          contain information of the 5 categories of a single promoter and its
          respective gene. 
    """
    name = epd_fasta_file[:-4] + "_" + GSE_RNA_seq_file[:-4]
    if os.path.isdir(name):
        pass
    else:
        os.mkdir(name)
    tsv_name = os.path.join(name, name + "_Root TRAP_mgd" + ".csv")
    epd_df_plants = epdparser(epd_fasta_file)
    epd_df_plants = epd_df_plants.rename({"Promoter_ID": "Ensembl_ID"}, axis=1)
    plant_rna_seq = pd.read_csv(open(GSE_RNA_seq_file), delimiter="\t")
    plant_rna_seq = plant_rna_seq.rename({"ID": "Ensembl_ID"}, axis=1)
    plant_matched_matrix = pd.merge(epd_df_plants, plant_rna_seq, on=["Ensembl_ID"])
    plant_sub_matched_matrix = plant_matched_matrix.iloc[:, :2]
    plant_sub_matched_matrix["logTPM"] = plant_matched_matrix.iloc[:, 8:10].mean(axis=1) + 1
    plant_sub_matched_matrix["logTPM"] = np.log2(plant_sub_matched_matrix["logTPM"])
    if os.path.isfile(tsv_name) == False:
        plant_sub_matched_matrix.to_csv(tsv_name)
    return plant_matched_matrix

In [49]:
plant_matched_matrix = wrapperparser_plant("plant_genes.txt", "GSE181536_TPM_data.txt")

In [35]:
plant_matched_matrix

,Ensembl_ID,Sequence,Shoot TRAP CK_rep1,Shoot TRAP CK_rep2,Shoot TRAP CK_rep3,Shoot TRAP MgD_rep1,Shoot TRAP MgD_rep2,Shoot TRAP MgD_rep3,Root TRAP CK_rep1,Root TRAP CK_rep2,Root TRAP CK_rep3,Root TRAP MgD_rep1,Root TRAP MgD_rep2,Root TRAP MgD_rep3,Shoot Total CK_rep1,Shoot Total CK_rep2,Shoot Total CK_rep3,Shoot Total MgD_rep1,Shoot Total MgD_rep2,Shoot Total MgD_rep3,Root Total CK_rep1,Root Total CK_rep2,Root Total CK_rep3,Root Total MgD_rep1,Root Total MgD_rep2,Root Total MgD_rep3
0,AT2G37860,TTAAAATCTTAAAATCTCCTTAAATCTTTCTAATAAATGATATTCT...,20.881660,19.841960,21.805930,22.421260,20.476790,24.296290,19.604850,17.051980,14.813890,14.479210,15.224900,11.935740,18.259560,19.244050,21.610400,23.975740,22.261550,24.037150,19.522410,18.372720,20.696530,21.615420,18.552870,24.577110
1,AT2G02050,CTTTCAACTGTAACTGTCTCTGTATCAATAACAACATTGCTGTGTA...,53.883224,60.623470,54.502460,52.146845,66.344416,50.589297,136.674327,139.219526,109.158527,121.860920,107.220401,109.783599,152.078937,139.481686,158.825857,145.097667,141.042307,148.303511,298.198841,243.925172,232.858565,251.048913,232.675169,243.214139
2,AT1G71810,TTCTTAAGTTTTAAACGTAATTATTATGAATAGTTTGGAATGATAC...,6.197430,6.782987,7.752321,8.245499,7.703395,8.082251,0.410788,0.562508,0.791142,0.368986,0.427580,0.879620,9.316288,9.655441,10.086710,11.535070,8.356302,8.765276,1.157464,2.333920,2.230266,1.952864,1.558832,2.616400
3,AT3G10860,TGTCTTGCAGACAAGGAACGTGGAAGATCTTCATCGTCTCCTTAGA...,48.308190,50.330180,49.577200,38.795160,39.503990,41.668920,121.188300,151.553800,132.498700,148.809100,122.585400,122.923100,188.522200,156.687700,170.364300,161.853700,172.512400,171.928500,430.619600,377.006900,354.428600,367.148800,369.989600,387.636800
4,AT5G44630,TTCAATTGCTAAGCCTATGCGGCCTTTACAACGAATCAGTTAAAAT...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060256,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.084258,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6378,AT5G56030,CAAGTGGTCAATCAAATCTTTTTTAGTTGGGCCCAAAATGTCTGTT...,756.846400,668.107200,489.813200,713.617100,514.066900,425.205800,439.237700,357.233100,459.464900,370.647000,312.092500,320.723300,442.969200,304.568300,337.276400,515.966200,430.108500,299.211000,312.523800,242.034600,277.155800,290.947300,211.932500,255.706700
6379,AT5G47770,AATGTTGAAGCACATGTTGTACTATAAAAAAACAGAGTTTATTATC...,35.749140,34.569810,40.627480,37.263740,40.147050,42.427730,73.656030,98.642250,100.463900,87.533250,100.107300,97.833460,53.170380,53.975620,47.653030,54.170010,48.274720,49.331080,171.955500,164.040100,174.114600,157.254000,160.498100,239.181200
6380,AT4G38520,GATGTCTCGTTCGGCGACTTGACCTCTCTCTCTCTCTCTCTTTTAC...,19.057040,24.856920,22.948830,30.486970,24.522990,23.138500,16.850990,21.382050,18.030960,18.122630,18.772230,18.570870,37.452100,36.999190,32.255900,37.672760,37.166600,31.649350,43.628750,50.011440,50.076010,44.444760,42.162730,41.317650
6381,AT1G07750,AAAAAAACGAAGAACGTGATGGATTAACAAAAATTATCTCCTCACA...,160.804288,167.541794,132.886379,161.809653,134.190695,149.972949,276.367142,157.550401,119.814395,188.936751,187.273150,137.966484,38.636050,35.954307,37.544831,36.793498,41.018319,38.606729,142.502709,74.353621,92.366791,94.720209,85.581285,100.636195


In [ ]:
epd_df_plants = epd_df_plants.rename({"Promoter_ID": "Ensembl_ID"}, axis=1)

In [ ]:
plant_rna_seq = pd.read_csv(open("GSE181536_TPM_data.txt"), delimiter="\t")
plant_rna_seq = plant_rna_seq.rename({"ID": "Ensembl_ID"}, axis=1)


In [ ]:
plant_matched_matrix = pd.merge(epd_df_plants, plant_rna_seq, on=["Ensembl_ID"])

In [ ]:
plantshoot_matched_matrix = plant_matched_matrix.iloc[:, :1]

In [ ]:
plantshoot_matched_matrix["logTPM"] = plant_matched_matrix.iloc[:, 2:4].mean(axis=1) + 1

In [ ]:
plantshoot_matched_matrix["logTPM"] = np.log2(plantshoot_matched_matrix["logTPM"])

normalization by distribution of data: add pseudocount 1 to TPM and FPKM columns, and perform log2 transformation
https://www-ncbi-nlm-nih-gov.ezproxy.library.wur.nl/pmc/articles/PMC6171491/

In [ ]:
import os

In [ ]:
ls

ENCFF292FVY.tsv          hg38_msxTm_ENCFF910TAZ/
ENCFF379CCS.tsv          hg38_msxTm.txt
ENCFF910TAZ.tsv          input_seq_promoter_motifs_locations/
example/                 input_seq_promoter_motifs_locations_plants/
find_motifs/             plant_genes.txt
hg38_msxTm_ENCFF292FVY/  replicate/
hg38_msxTm_ENCFF379CCS/


In [ ]:
import seaborn as sns
sns.distplot(final_matrix["TPM"]).set(xlim=(0))

NameError: ignored

In [ ]:
final_matrix["FPKM"].plot.kde()

In [ ]:
final_matrix["logTPM"].plot.kde()

In [ ]:
final_matrix["logFPKM"].plot.kde()

In [ ]:
final_matrix.to_csv('hg38_msxTm_ENCFF379CCS.csv') 